# Assignment #6: Dependency parsing
Author: Pierre Nugues

## Objectives

This assignment is inspired by the CoNLL 2018 shared task of the conference on computational natural language learning on dependency parsing, http://universaldependencies.org/conll18/. It is a follower of <a href="http://ilk.uvt.nl/conll/">CONLL-X</a>, which was the first large-scale evaluation of dependency parsers.
            
In this session, you will implement a dependency parser for Swedish and, optionally, another language that you will choose.

The objectives of this assignment are to:
* Know what a dependency graph is
* Understand the principles of a transition-based parser
* Extend the parser with a guiding predicate that parses an annotated dependency graph
* Extract features to learn parsing actions from an annotated corpus
* Write a short report on your results

## Organization and location

You can work alone or collaborate with another student.
Each group will have to:
* Write a program that parses a sentence when the dependency graph is known
* Extract features from the parsing actions.
* Train a classifier
* Apply it on a test corpus
* Evaluate the results

## Corpora

As corpora, you will use the Universal Dependencies: https://universaldependencies.org/. The corpora are the same that you used in the 5th assignment, but you will use the test set in addition to the training set. 
1. You will train your parser on training part of the Swedish _Talbanken_ corpus and you will evaluate it on the test set. 
2. Optionally, you will repeat the experiment with another language. 
3. You will comment your results in the report. 

You will load the corpora as in the 5th assignment and the next cells are just copies from this assignment.

### Corpus location

Here are the corpus locations you will use. You may have to adjust `ud_path`.

In [1]:
import os
import copy
ud_path = 'corpus/ud-treebanks-v2.6/'

In [2]:
path_sv_train = ud_path + 'UD_Swedish-Talbanken/sv_talbanken-ud-train.conllu'
path_sv_test = ud_path + 'UD_Swedish-Talbanken/sv_talbanken-ud-test.conllu'
path_eng_train = ud_path + 'UD_English-EWT/en_ewt-ud-train.conllu' 
path_eng_test = ud_path + 'UD_English-EWT/en_ewt-ud-test.conllu'

The column names of the CoNLL-U corpora

In [3]:
column_names_u = ['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC']

#### Functions to read the CoNLL-U files

In [4]:
def read_sentences(file):
    """
    Creates a list of sentences from the corpus
    Each sentence is a string
    :param file:
    :return:
    """
    f = open(file, encoding='utf-8').read().strip()
    sentences = f.split('\n\n')
    return sentences

In [5]:
def split_rows(sentences, column_names):
    """
    Creates a list of sentence where each sentence is a list of lines
    Each line is a dictionary of columns
    :param sentences:
    :param column_names:
    :return:
    """
    new_sentences = []
    root_values = ['0', 'root', 'root', 'root', 'root', 'root', '0', 'root', 'root', 'root']
    start = [dict(zip(column_names, root_values))]
    for sentence in sentences:
        rows = sentence.split('\n')
        sentence = [dict(zip(column_names, row.split('\t'))) for row in rows if row[0] != '#']
        sentence = start + sentence
        new_sentences.append(sentence)
    return new_sentences

#### Reading the corpus

We load the Swedish _Talbanken_ corpus.

In [6]:
sentences = read_sentences(path_sv_train)
formatted_corpus_train = split_rows(sentences, column_names_u)

In [7]:
len(formatted_corpus_train)

4303

The parsed sentence: _Individuell beskattning av arbetsinkomster_

In [8]:
formatted_corpus_train[0]

[{'ID': '0',
  'FORM': 'root',
  'LEMMA': 'root',
  'UPOS': 'root',
  'XPOS': 'root',
  'FEATS': 'root',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': 'root',
  'MISC': 'root'},
 {'ID': '1',
  'FORM': 'Individuell',
  'LEMMA': 'individuell',
  'UPOS': 'ADJ',
  'XPOS': 'JJ|POS|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing',
  'HEAD': '2',
  'DEPREL': 'amod',
  'DEPS': '2:amod',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'beskattning',
  'LEMMA': 'beskattning',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': '0:root',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'av',
  'LEMMA': 'av',
  'UPOS': 'ADP',
  'XPOS': 'PP',
  'FEATS': '_',
  'HEAD': '4',
  'DEPREL': 'case',
  'DEPS': '4:case',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'arbetsinkomster',
  'LEMMA': 'arbetsinkomst',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|PLU|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|G

#### Removing indices that are not integers

To ease the processing of some corpora, we remove the indices which are not integers. We do this because `ID` is not necessarily a number.

In [9]:
def clean_indicies(formatted_corpus):
    formatted_corpus_clean = []
    for sentence in formatted_corpus:
        formatted_corpus_clean.append([word for word in sentence if word['ID'].isdigit()])
    return formatted_corpus_clean          

In [10]:
formatted_corpus_train_clean = clean_indicies(formatted_corpus_train)
formatted_corpus_train_clean[0]

[{'ID': '0',
  'FORM': 'root',
  'LEMMA': 'root',
  'UPOS': 'root',
  'XPOS': 'root',
  'FEATS': 'root',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': 'root',
  'MISC': 'root'},
 {'ID': '1',
  'FORM': 'Individuell',
  'LEMMA': 'individuell',
  'UPOS': 'ADJ',
  'XPOS': 'JJ|POS|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing',
  'HEAD': '2',
  'DEPREL': 'amod',
  'DEPS': '2:amod',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'beskattning',
  'LEMMA': 'beskattning',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': '0:root',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'av',
  'LEMMA': 'av',
  'UPOS': 'ADP',
  'XPOS': 'PP',
  'FEATS': '_',
  'HEAD': '4',
  'DEPREL': 'case',
  'DEPS': '4:case',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'arbetsinkomster',
  'LEMMA': 'arbetsinkomst',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|PLU|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|G

## Transition parser

For each sentence with a projective dependency graph, there is an action sequence that enables the transition parser
to generate this graph. Gold standard parsing corresponds to the sequence of parsing actions, left-arc (<tt>la</tt>), right-arc (<tt>ra</tt>), shift (<tt>sh</tt>), and reduce (<tt>re</tt>) that produces the manually-obtained, gold standard, graph.

### The transitions

Here are implementations of the parsing transitions. Read them and be sure you understand them.

In [11]:
def shift(stack, queue, graph):
    """
    Shift the first word in the queue onto the stack
    :param stack:
    :param queue:
    :param graph:
    :return:
    """
    stack = [queue[0]] + stack
    queue = queue[1:]
    return stack, queue, graph


def reduce(stack, queue, graph):
    """
    Remove the first item from the stack
    :param stack:
    :param queue:
    :param graph:
    :return:
    """
    return stack[1:], queue, graph


def right_arc(stack, queue, graph, deprel=False):
    """
    Creates an arc from the top of the stack to the first in the queue
    and shifts
    The deprel argument is either read from the manually-annotated corpus
    (deprel=False) or assigned by the parser. In this case, the deprel
    argument has a value
    :param stack:
    :param queue:
    :param graph:
    :param deprel: either read from the manually-annotated corpus (value false)
    or assigned by the parser
    :return:
    """
    graph['heads'][queue[0]['ID']] = stack[0]['ID']
    if deprel:
        graph['deprels'][queue[0]['ID']] = deprel
    else:
        graph['deprels'][queue[0]['ID']] = queue[0]['DEPREL']
    # If we create an arc from the 'root', we introduce a statement to pop it to avoid multiple roots
    if stack[0]['ID'] == '0':
        stack = stack[1:]
    return shift(stack, queue, graph)


def left_arc(stack, queue, graph, deprel=False):
    """
    Creates an arc from the first in the queue to the top of the stack
    and reduces it.
    The deprel argument is either read from the manually-annotated corpus
    (deprel=False) or assigned by the parser. In this case, the deprel
    argument has a value
    :param stack:
    :param queue:
    :param graph:
    :param deprel: either read from the manually-annotated corpus (value false)
    or assigned by the parser
    :return:
    """
    graph['heads'][stack[0]['ID']] = queue[0]['ID']
    if deprel:
        graph['deprels'][stack[0]['ID']] = deprel
    else:
        graph['deprels'][stack[0]['ID']] = stack[0]['DEPREL']        
    return reduce(stack, queue, graph)

### Constrains on the transitions

We add a few constraints before we carry out the transitions. Given a manually-annotated dependency graph, look at the conditions (`can_...()` functions) on the stack and the current input list -- the queue -- to execute left-arc, right-arc, shift, or reduce.

In [12]:
def can_reduce(stack, graph):
    """
    Checks that the top of the stack has a head
    :param stack:
    :param graph:
    :return:
    """
    if not stack:
        return False
    if stack[0]['ID'] in graph['heads']:
        return True
    else:
        return False

    
def can_leftarc(stack, graph):
    """
    Checks that the top of the has no head
    :param stack:
    :param graph:
    :return:
    """
    if not stack:
        return False
    if stack[0]['ID'] in graph['heads']:
        return False
    else:
        return True


def can_rightarc(stack):
    """
    Simply checks there is a stack
    :param stack:
    :return:
    """
    if not stack:
        return False
    else:
        return True
    
def can_assign_root(graph):
    return not exists_root(graph);

### Finding the transitions from a manually-parsed sentence

Using an annotated corpus, we can derive the action sequences producing the manually-parsed sentences (provided that they are projective). We use an oracle for this as explained during the lectures.

In [13]:
def oracle(stack, queue, graph):
    """
    Gold standard parsing
    Produces a sequence of transitions from a manually-annotated corpus:
    sh, re, ra.deprel, la.deprel
    :param stack: The stack
    :param queue: The input list
    :param graph: The set of relations already parsed
    :return: the transition and the grammatical function (deprel) in the
    form of transition.deprel
    """
    # Right arc
    if stack and stack[0]['ID'] == queue[0]['HEAD']:
        # print('ra', queue[0]['DEPREL'], stack[0]['UPOS'], queue[0]['UPOS'])
        deprel = '.' + queue[0]['DEPREL']
        stack, queue, graph = right_arc(stack, queue, graph)
        return stack, queue, graph, 'ra' + deprel
    # Left arc
    if stack and queue[0]['ID'] == stack[0]['HEAD']:
        # print('la', stack[0]['DEPREL'], stack[0]['UPOS'], queue[0]['UPOS'])
        deprel = '.' + stack[0]['DEPREL']
        stack, queue, graph = left_arc(stack, queue, graph)
        return stack, queue, graph, 'la' + deprel
    # Reduce
    if stack and can_reduce(stack, graph):
        for word in stack:
            if (word['ID'] == queue[0]['HEAD'] or
                    word['HEAD'] == queue[0]['ID']):
                # print('re', stack[0]['UPOS'], queue[0]['UPOS'])
                stack, queue, graph = reduce(stack, queue, graph)
                return stack, queue, graph, 're'
    # Shift
    # print('sh', [], queue[0]['UPOS'])
    stack, queue, graph = shift(stack, queue, graph)
    return stack, queue, graph, 'sh'

### Dealing with nonprojective graphs

Oracle parsing produces a sequence of transitions if the graph is projective and well-formed. If not, we will have headless words in the stack. Parsing normally terminates when the queue is empty. We also empty the stack to be sure that all the words have a head. We attach headless words to the root word of the sentence.

In [14]:
def exists_root(graph):
    for (x, y)  in graph['heads'].items():
        if y == '0' and x != '0':
            return x
    return False

In [15]:
def empty_stack(stack, graph):
    """
    Pops the items in the stack. If they have no head, they are assigned
    a ROOT head
    :param stack:
    :param graph:
    :return:
    """
    idx_root = exists_root(graph)
    # There is already a root
    if idx_root:
        for word in stack:
            if word['ID'] not in graph['heads']:
                graph['heads'][word['ID']] = idx_root
                graph['deprels'][word['ID']] = 'dep'
    else:
        # There is no root. We assign the root to the first headless word.
        for word in stack:
            if word['ID'] not in graph['heads']:
                if idx_root:
                    graph['heads'][word['ID']] = idx_root
                    graph['deprels'][word['ID']] = 'dep'
                else:
                    graph['heads'][word['ID']] = '0'
                    graph['deprels'][word['ID']] = 'root'
                    idx_root = word['ID']
    stack = []
    return stack, graph

### Checking if two graphs are equal

The `equal_graphs()` utility checks if the graph obtained from a sequence of transitions is equal to the annotated graph. It is normally the case, except with nonprojective graphs.

In [16]:
def equal_graphs(sentence, graph, verbose=False):
    """
    Checks that the graph corresponds to the gold standard annotation of a sentence
    :param sentence:
    :param graph:
    :return:
    """
    equal = True
    for word in sentence:
        if word['ID'] in graph['heads'] and word['HEAD'] == graph['heads'][word['ID']]:
            pass
        else:
            equal = False
            if verbose:
                print(word, flush=True)
    return equal

In [117]:
def equal_graphs_2(sentence, graph, cond_deprel, verbose):
    """
    Checks that the graph corresponds to the gold standard annotation of a sentence
    :param sentence:
    :param graph:
    :return:
    """
    equal = True
    for word in sentence:
        if word['ID'] in graph['heads']:
            if word['HEAD'] == graph['heads'][word['ID']]:
                pass
            else:
                if cond_deprel:
                    equal = not (graph['deprels'][word['ID']] == 'dep')
                else:
                    word['Error'] = 'Wrong head in graph'
                    equal = False
        else:
            word['Error'] = 'Missing ID in graph'
            equal = False
            if verbose:
                print(word, flush=True)
    return equal

In [18]:
#sentence = formatted_corpus_test_clean[0]
#sentence[1]['HEAD'] = '5'
#graph = {'heads' : {'0' : '0', '2' : '5', '2' : '3', '3' : '7', '4' : '7', '5' : '7', '6' : '7', '7' : '0', '8' : '7', '9' : '7', '10' : '11', '11' : '7', '12' : '7', '13' : '7', '14' : '7', '15' : '7'}}
#graph_collection_errors([sentence], [graph])


In [116]:
def graph_collection_errors(sentences, graphs, ret_smalest=True, cond_deprel=False, verbose=False):
    sentences = copy.deepcopy(sentences)
    smallest = 100;
    error_graphs = []
    
    for i, sentence in enumerate(sentences):
        if not equal_graphs_2(sentence, graphs[i], cond_deprel, verbose):
            if not ret_smalest:
                error_graphs.append(sentence);
            elif len(sentence) < smallest: 
                smallest = len(sentence)
                error_graphs = sentence
    return error_graphs

### Parsing an annotated corpus with an oracle

You will now run the code below. With it, you will produce a sequence of transitions for each sentence. If the graph is projective, applying the sequence to the sentence will recreate the gold-standard annotation.

For this experiment:
1. Understand from the slides used during the lecture how the oracle carries out a gold standard parsing. 
2. The parser can only deal with projective sentences. In the case of a nonprojective one, the parsed graph and the manually-annotated sentence are not equal. Examine one nonprojective sentence (just set `verbose`to `True` in the code below) and explain why it is not projective. Take a short one (the shortest). You will **describe** this in the report.

In [20]:
def init_config(sentence):
    stack = []
    queue = list(sentence)
    graph = {}
    graph['heads'] = {}
    graph['heads']['0'] = '0'
    graph['deprels'] = {}
    graph['deprels']['0'] = 'ROOT'
    return stack, queue, graph

In [21]:
verbose = True
projectivization = False

transition_corpus = []
graph_corpus = []

small = 10000
sent = {}
gr = {}
count = 0

for sent_cnt, sentence in enumerate(formatted_corpus_train_clean):
    #print(sentence)
    stack, queue, graph = init_config(sentence)
    transition_sent = []
    while queue:
        stack, queue, graph, trans = oracle(stack, queue, graph)
        transition_sent.append(trans)
    stack, graph = empty_stack(stack, graph)
    transition_corpus.append(transition_sent)
    graph_corpus.append(graph)

    if verbose:
        if not equal_graphs(sentence, graph):
            length = len(sentence)
            
            if(length < small):
                small = length
                sent = sentence
                gr = graph
                count += 1
                
            print('Annotation and gold-standard parsing not equal')
            print('Sentence:', sentence)
            print('Gold-standard graph', graph)
    # Poorman's projectivization to have well-formed graphs.
    # We just just assign the same heads as what gold standard parsing did
    # This guarantee a projective sentence
    if projectivization:
        for word in sentence:
            word['HEAD'] = graph['heads'][word['ID']]
print('\nProcessed ' + str(sent_cnt) + ' sentences')

Annotation and gold-standard parsing not equal
Sentence: [{'ID': '0', 'FORM': 'root', 'LEMMA': 'root', 'UPOS': 'root', 'XPOS': 'root', 'FEATS': 'root', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': 'root', 'MISC': 'root'}, {'ID': '1', 'FORM': 'Kommunalt', 'LEMMA': 'kommunal', 'UPOS': 'ADJ', 'XPOS': 'JJ|POS|NEU|SIN|IND|NOM', 'FEATS': 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing', 'HEAD': '2', 'DEPREL': 'amod', 'DEPS': '2:amod', 'MISC': '_'}, {'ID': '2', 'FORM': 'grundavdrag', 'LEMMA': 'grundavdrag', 'UPOS': 'NOUN', 'XPOS': 'NN|NEU|SIN|IND|NOM', 'FEATS': 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing', 'HEAD': '4', 'DEPREL': 'nsubj:pass', 'DEPS': '4:nsubj:pass', 'MISC': '_'}, {'ID': '3', 'FORM': 'kan', 'LEMMA': 'kunna', 'UPOS': 'AUX', 'XPOS': 'VB|PRS|AKT', 'FEATS': 'Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act', 'HEAD': '4', 'DEPREL': 'aux', 'DEPS': '4:aux', 'MISC': '_'}, {'ID': '4', 'FORM': 'avräknas', 'LEMMA': 'avräkna', 'UPOS': 'VERB', 'XPOS': 'VB|PRS|SFO', 'FEATS': 'Mood=Ind|Tens

Annotation and gold-standard parsing not equal
Sentence: [{'ID': '0', 'FORM': 'root', 'LEMMA': 'root', 'UPOS': 'root', 'XPOS': 'root', 'FEATS': 'root', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': 'root', 'MISC': 'root'}, {'ID': '1', 'FORM': 'Skaffa', 'LEMMA': 'skaffa', 'UPOS': 'VERB', 'XPOS': 'VB|IMP|AKT', 'FEATS': 'Mood=Imp|VerbForm=Fin|Voice=Act', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': '0:root', 'MISC': '_'}, {'ID': '2', 'FORM': 'för', 'LEMMA': 'för', 'UPOS': 'ADP', 'XPOS': 'PP', 'FEATS': '_', 'HEAD': '3', 'DEPREL': 'case', 'DEPS': '3:case', 'MISC': '_'}, {'ID': '3', 'FORM': 'säkerhets', 'LEMMA': 'säkerhet', 'UPOS': 'NOUN', 'XPOS': 'NN|UTR|SIN|IND|GEN', 'FEATS': 'Case=Gen|Definite=Ind|Gender=Com|Number=Sing', 'HEAD': '1', 'DEPREL': 'obl', 'DEPS': '1:obl:för_skull', 'MISC': '_'}, {'ID': '4', 'FORM': 'skull', 'LEMMA': 'skull', 'UPOS': 'NOUN', 'XPOS': 'NN|UTR|SIN|IND|NOM', 'FEATS': 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing', 'HEAD': '2', 'DEPREL': 'fixed', 'DEPS': '2:fixed', 'MISC': '_'}

Annotation and gold-standard parsing not equal
Sentence: [{'ID': '0', 'FORM': 'root', 'LEMMA': 'root', 'UPOS': 'root', 'XPOS': 'root', 'FEATS': 'root', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': 'root', 'MISC': 'root'}, {'ID': '1', 'FORM': 'Det', 'LEMMA': 'den', 'UPOS': 'PRON', 'XPOS': 'PN|NEU|SIN|DEF|SUB/OBJ', 'FEATS': 'Definite=Def|Gender=Neut|Number=Sing|PronType=Prs', 'HEAD': '2', 'DEPREL': 'expl', 'DEPS': '2:expl', 'MISC': '_'}, {'ID': '2', 'FORM': 'kostar', 'LEMMA': 'kosta', 'UPOS': 'VERB', 'XPOS': 'VB|PRS|AKT', 'FEATS': 'Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': '0:root', 'MISC': '_'}, {'ID': '3', 'FORM': 'cirka', 'LEMMA': 'cirka', 'UPOS': 'ADV', 'XPOS': 'AB', 'FEATS': '_', 'HEAD': '4', 'DEPREL': 'advmod', 'DEPS': '4:advmod', 'MISC': '_'}, {'ID': '4', 'FORM': '20', 'LEMMA': '20', 'UPOS': 'NUM', 'XPOS': 'RG|NOM', 'FEATS': 'Case=Nom|NumType=Card', 'HEAD': '5', 'DEPREL': 'nummod', 'DEPS': '5:nummod', 'MISC': '_'}, {'ID': '5', 'FORM': 'kronor', 

Annotation and gold-standard parsing not equal
Sentence: [{'ID': '0', 'FORM': 'root', 'LEMMA': 'root', 'UPOS': 'root', 'XPOS': 'root', 'FEATS': 'root', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': 'root', 'MISC': 'root'}, {'ID': '1', 'FORM': 'Det', 'LEMMA': 'den', 'UPOS': 'PRON', 'XPOS': 'PN|NEU|SIN|DEF|SUB/OBJ', 'FEATS': 'Definite=Def|Gender=Neut|Number=Sing|PronType=Prs', 'HEAD': '2', 'DEPREL': 'nsubj', 'DEPS': '2:nsubj', 'MISC': '_'}, {'ID': '2', 'FORM': 'innebär', 'LEMMA': 'innebära', 'UPOS': 'VERB', 'XPOS': 'VB|PRS|AKT', 'FEATS': 'Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': '0:root', 'MISC': '_'}, {'ID': '3', 'FORM': 'att', 'LEMMA': 'att', 'UPOS': 'SCONJ', 'XPOS': 'SN', 'FEATS': '_', 'HEAD': '8', 'DEPREL': 'mark', 'DEPS': '8:mark', 'MISC': '_'}, {'ID': '4', 'FORM': 'matematikundervisningen', 'LEMMA': 'matematikundervisning', 'UPOS': 'NOUN', 'XPOS': 'NN|UTR|SIN|DEF|NOM', 'FEATS': 'Case=Nom|Definite=Def|Gender=Com|Number=Sing', 'HEAD': '8', 'DEPREL'

Annotation and gold-standard parsing not equal
Sentence: [{'ID': '0', 'FORM': 'root', 'LEMMA': 'root', 'UPOS': 'root', 'XPOS': 'root', 'FEATS': 'root', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': 'root', 'MISC': 'root'}, {'ID': '1', 'FORM': 'Man', 'LEMMA': 'man', 'UPOS': 'PRON', 'XPOS': 'PN|UTR|SIN|IND|SUB', 'FEATS': 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|PronType=Ind', 'HEAD': '4', 'DEPREL': 'nsubj', 'DEPS': '4:nsubj', 'MISC': '_'}, {'ID': '2', 'FORM': 'måste', 'LEMMA': 'måste', 'UPOS': 'AUX', 'XPOS': 'VB|PRS|AKT', 'FEATS': 'Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act', 'HEAD': '4', 'DEPREL': 'aux', 'DEPS': '4:aux', 'MISC': '_'}, {'ID': '3', 'FORM': 'alltid', 'LEMMA': 'alltid', 'UPOS': 'ADV', 'XPOS': 'AB', 'FEATS': '_', 'HEAD': '4', 'DEPREL': 'advmod', 'DEPS': '4:advmod', 'MISC': '_'}, {'ID': '4', 'FORM': 'räkna', 'LEMMA': 'räkna', 'UPOS': 'VERB', 'XPOS': 'VB|INF|AKT', 'FEATS': 'VerbForm=Inf|Voice=Act', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': '0:root', 'MISC': '_'}, {'ID': '5', 'FORM'

Annotation and gold-standard parsing not equal
Sentence: [{'ID': '0', 'FORM': 'root', 'LEMMA': 'root', 'UPOS': 'root', 'XPOS': 'root', 'FEATS': 'root', 'HEAD': '0', 'DEPREL': 'root', 'DEPS': 'root', 'MISC': 'root'}, {'ID': '1', 'FORM': 'Kärlek', 'LEMMA': 'kärlek', 'UPOS': 'NOUN', 'XPOS': 'NN|UTR|SIN|IND|NOM', 'FEATS': 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing', 'HEAD': '10', 'DEPREL': 'obj', 'DEPS': '10:obj', 'MISC': '_'}, {'ID': '2', 'FORM': 'i', 'LEMMA': 'i', 'UPOS': 'ADP', 'XPOS': 'PP', 'FEATS': '_', 'HEAD': '4', 'DEPREL': 'case', 'DEPS': '4:case', 'MISC': '_'}, {'ID': '3', 'FORM': 'samma', 'LEMMA': 'samma', 'UPOS': 'DET', 'XPOS': 'DT|UTR/NEU|SIN/PLU|IND', 'FEATS': 'Definite=Ind|PronType=Ind', 'HEAD': '4', 'DEPREL': 'det', 'DEPS': '4:det', 'MISC': '_'}, {'ID': '4', 'FORM': 'riktning', 'LEMMA': 'riktning', 'UPOS': 'NOUN', 'XPOS': 'NN|UTR|SIN|IND|NOM', 'FEATS': 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing', 'HEAD': '1', 'DEPREL': 'nmod', 'DEPS': '1:nmod:i', 'MISC': '_'}, {'ID': '

### Checking gold-standard parsing

Apply manually the transition sequence you obtained to the first sentence and check that it parses it correctly. You will draw a stack and a queue with **seven steps** and you will describe this in your report

In [22]:
def getText(sentence):
    text = ""
    
    for word in sentence:
        text += word['FORM'] + " "
        
    return text       

## Training a classifier

We can now train a classifier to predict an action from a current parsing context. To be able to predict the next action from a given parsing state, gold standard parsing must also extract feature vectors at each step of the parsing procedure. The simplest parsing context corresponds to words' part of speech on the top of the stack and head of the input list (the queue).
    
Once the data collected, the training procedure will produce a 4-class classifier that you will embed in
Nivre's parser to choose the next action. During parsing, Nivre's parser will call the classifier to choose
the next action in the set {<tt>la</tt>, <tt>ra</tt>, <tt>sh</tt>, <tt>re</tt>} using the current context.

You will use two feature sets to build your models:
* The top of the stack and the first word of the input list (word forms and parts of speech);
* The two first words and POS on the top of the stack and the two first words and POS of the input list;
* You will also add constraints to actions. You will encode these constraints as Boolean features.

### Parsing the grammatical functions

Using the actions in the set {<tt>la</tt>, <tt>ra</tt>, <tt>sh</tt>, <tt>re</tt>} produces an unlabelled
graph. It is easy to extend the parser so that it can label the graph with grammatical functions. In this
case, we must complement the actions <tt>la</tt>
and <tt>ra</tt> with their function using this notation for example:<tt>la.mod</tt>, <tt>la.case</tt>, <tt>ra.nmod</tt>, etc. where the prefix is the action and the suffix is the function.

### Extracting features 

The final goal is to parse the Swedish corpus and produce a labelled dependency graph. 

You will consider two feature sets and you will train the corresponding logistic regression models using scikit-learn:
1. The first set will use the word and the part of speech extracted from the first element in the stack and the first in the queue,
2. the second one will use two elements from the stack and two from the input list.

These sets will include two additional Boolean parameters, "can do left arc" and "can do reduce", which will model constraints on the parser's actions. In total, the feature sets will then have six, respectively ten parameters.

This means that the purpose of this assignment is to generate two scikit-learn models for the labelled graphs. We use the depth parameter for this: The depth of the stack and the queue, either 1 or 2. Start with 1.

In [23]:
depth = 2

You will need the `queue_stack()` function.

In [24]:
def queue_stack(queue_or_stack, graph, depth, pos=True, lex=True):
    features = []
    features_pos = ['nil'] * depth
    features_lex = ['nil'] * depth
    features_deprel = ['nil'] * depth
    if queue_or_stack:
        for i, word in list(enumerate(queue_or_stack))[:depth]:
            features_pos[i] = queue_or_stack[i]['UPOS']
            features_lex[i] = queue_or_stack[i]['FORM']
    if pos:
        features += features_pos
    if lex:
        features += features_lex
    return features

Optionally, you may want to extend the feature vector with words to the left of the top of the stack with the `right_context()` function. If the top of the stack has index $i$, you will extract the words and their parts of speech at index $i + 1$, $i+2$. This will noticeably improve the performance.

In [25]:
def right_context(stack, sentence, depth, pos=True, lex=True):
    features = []
    features_pos = ['nil'] * depth
    features_lex = ['nil'] * depth
    if stack:
        fw_id = int(stack[0]['ID']) + 1
        for i, word in list(enumerate(sentence))[fw_id: fw_id + depth]:
            features_pos[i - fw_id] = sentence[i]['UPOS']
            features_lex[i - fw_id] = sentence[i]['FORM']
    if pos:
        features += features_pos
    if lex:
        features += features_lex
    return features

The next function returns the features in a dictionary format compatible with scikit-learn. You have a code example of feature encoding in this format in the chunking program.

In [26]:
def extract(depth, stack, queue, graph, sentence):
    """
    :param stack:
    :param queue:
    :param graph:
    :param feature_names:
    :param sentence:
    :return:
    """
    improved = True
    if improved:
        x = (queue_stack(stack, graph, depth) +
             queue_stack(queue, graph, depth) +
             right_context(stack, sentence, depth) +
             [can_reduce(stack, graph), can_leftarc(stack, graph), can_assign_root(graph)])
    else:
        x = (queue_stack(stack, graph, depth) +
             queue_stack(queue, graph, depth) +
             [can_reduce(stack, graph), can_leftarc(stack, graph)])     
    feature_names = ['feat' + str(i) for i in range(len(x))]
    features = dict(zip(feature_names, x))
    return features

Now write a loop to parse the annotated corpus using the oracle and collect the features in a matrix ($\mathbf{X}$) and the transitions in a vector ($\mathbf{y}$). 

The first lines of your features for the 4 parameters ($\mathbf{x}$) and labelled actions ($y$) should look like the excerpt below, where the columns correspond to stack0_POS, stack1_POS, stack0_word, stack1_word, queue0_POS, queue1_POS, queue0_word, queue1_word, can-re, can-la, and the transition value (`depth = 2`):
$\mathbf{X} =
\begin{bmatrix}
\text{nil}& \text{nil} &\text{nil} & \text{nil} & \text{ROOT} & \text{ADJ} & \text{ROOT} & \text{Individuell} & \text{False} & \text{False}\\
\text{ROOT} &     \text{nil} &     \text{ROOT} &     \text{nil} &     \text{ADJ} &     \text{NOUN} &     \text{Individuell} &     \text{beskattning} &     \text{True} &     \text{False}\\ 
\text{ADJ} &     \text{ROOT} &     \text{Individuell} &     \text{ROOT} &     \text{NOUN} &     \text{ADP} &     \text{beskattning} &     \text{av} &     \text{False} &     \text{True}\\ 
\text{ROOT} &     \text{nil} &     \text{ROOT} &     \text{nil} &     \text{NOUN} &     \text{ADP} &     \text{beskattning} &     \text{av} &     \text{True} &     \text{False}\\
\text{NOUN} &     \text{ROOT} &     \text{beskattning} &     \text{ROOT} &     \text{ADP} &     \text{NOUN} &     \text{av} &     \text{arbetsinkomster} &     \text{True} &     \text{False}\\
\text{ADP} &     \text{NOUN} &     \text{av} &     \text{beskattning} &     \text{NOUN} &     \text{nil} &     \text{arbetsinkomster} &     \text{nil} &     \text{False} &     \text{True}\\  \text{NOUN} &     \text{ROOT} &     \text{beskattning} &     \text{ROOT} &     \text{NOUN} &     \text{nil} &     \text{arbetsinkomster} &     \text{nil} &     \text{True} &  \text{False}
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
\text{sh}\\
\text{sh}\\
\text{la.amod}\\
\text{ra.root}\\
\text{sh}\\
\text{la.case}\\
\text{ra.nmod}
\end{bmatrix}$

You will store your matrix in a Python dictionary and the classes in a list

In [27]:
X_dict = []
y_symbols = []

In [28]:
# Write your code here

for sentence in formatted_corpus_train_clean:  
    stack, queue, graph = init_config(sentence) 

    while queue:
        feature_vector = extract(depth, stack, queue, graph, sentence)
        X_dict.append(feature_vector)   
        
        stack, queue, graph, instruction = oracle(stack, queue, graph)        
        y_symbols.append(instruction)

In [29]:
X_dict[:7]

[{'feat0': 'nil',
  'feat1': 'nil',
  'feat2': 'nil',
  'feat3': 'nil',
  'feat4': 'root',
  'feat5': 'ADJ',
  'feat6': 'root',
  'feat7': 'Individuell',
  'feat8': 'nil',
  'feat9': 'nil',
  'feat10': 'nil',
  'feat11': 'nil',
  'feat12': False,
  'feat13': False,
  'feat14': True},
 {'feat0': 'root',
  'feat1': 'nil',
  'feat2': 'root',
  'feat3': 'nil',
  'feat4': 'ADJ',
  'feat5': 'NOUN',
  'feat6': 'Individuell',
  'feat7': 'beskattning',
  'feat8': 'ADJ',
  'feat9': 'NOUN',
  'feat10': 'Individuell',
  'feat11': 'beskattning',
  'feat12': True,
  'feat13': False,
  'feat14': True},
 {'feat0': 'ADJ',
  'feat1': 'root',
  'feat2': 'Individuell',
  'feat3': 'root',
  'feat4': 'NOUN',
  'feat5': 'ADP',
  'feat6': 'beskattning',
  'feat7': 'av',
  'feat8': 'NOUN',
  'feat9': 'ADP',
  'feat10': 'beskattning',
  'feat11': 'av',
  'feat12': False,
  'feat13': True,
  'feat14': True},
 {'feat0': 'root',
  'feat1': 'nil',
  'feat2': 'root',
  'feat3': 'nil',
  'feat4': 'NOUN',
  'feat5': '

In [30]:
y_symbols[:7]

['sh', 'sh', 'la.amod', 'ra.root', 'sh', 'la.case', 'ra.nmod']

### Fitting the models

Generate the two scikit-learn models using the code models from the chunking labs.

Vectorize your `X_dict` into an `X` matrix using `DictVectorizer()`

In [31]:
# Write your code here
from sklearn.feature_extraction import DictVectorizer
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn import tree
vec = DictVectorizer()
X = vec.fit_transform(X_dict);

Fit the model. With sklearn, you can use `y_symbols` directly. Use `verbose=True` and `n_jobs=8` or more.

In [32]:
y_symbols

['sh',
 'sh',
 'la.amod',
 'ra.root',
 'sh',
 'la.case',
 'ra.nmod',
 'sh',
 'sh',
 'la.case',
 'sh',
 'la.obl',
 'ra.root',
 'sh',
 'la.amod',
 'ra.nsubj:pass',
 'ra.punct',
 're',
 'ra.appos',
 're',
 'ra.punct',
 'sh',
 'la.case',
 're',
 'ra.nmod',
 're',
 're',
 'ra.punct',
 'sh',
 'sh',
 'la.nsubj',
 'ra.root',
 'ra.advmod',
 'sh',
 'sh',
 'la.advmod',
 'sh',
 'la.nummod',
 'sh',
 'sh',
 'sh',
 'la.mark',
 'la.aux',
 'la.nsubj',
 'la.mark',
 're',
 'ra.ccomp',
 'sh',
 'la.case',
 'ra.obl',
 'sh',
 'sh',
 'la.amod',
 'la.case',
 'ra.nmod',
 're',
 're',
 're',
 'ra.punct',
 'sh',
 'sh',
 'la.nsubj',
 'ra.root',
 'ra.xcomp',
 'sh',
 'sh',
 'la.det',
 'la.case',
 're',
 'ra.obl',
 'sh',
 'la.case',
 're',
 'ra.obl',
 're',
 'ra.punct',
 'sh',
 'sh',
 'la.nsubj',
 'ra.root',
 'ra.advmod',
 'sh',
 'sh',
 'la.amod',
 'la.case',
 're',
 'ra.obl',
 'sh',
 'la.cc',
 'ra.conj',
 're',
 're',
 'ra.punct',
 'sh',
 'sh',
 'la.det',
 'sh',
 'sh',
 'sh',
 'sh',
 'la.det',
 'la.case',
 'sh',
 'l

In [33]:
# Write your code here
classifier = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes=(15,), alpha=0.001)
model = classifier.fit(X, y_symbols)

/home/viktor/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## Prediction

Now use this model to predict the sentences in the test corpus

In [74]:
sentences_test = read_sentences(path_sv_test)
formatted_corpus_test = split_rows(sentences_test, column_names_u)
formatted_corpus_test_clean = clean_indicies(formatted_corpus_test)
clean_test = copy.deepcopy(formatted_corpus_test_clean)

In [35]:
def apply_transition(stack, queue, graph, trans):
    if stack and trans[:2] == 'ra':
        if trans[3:] == 'root' and exists_root(graph):
            stack, queue, graph = shift(stack, queue, graph)
            return stack, queue, graph, 'sh'
        else:
            stack, queue, graph = right_arc(stack, queue, graph, trans[3:])
            return stack, queue, graph, 'ra'
    if stack and can_leftarc(stack, graph) and trans[:2] == 'la':
        if trans[3:] == 'root' and exists_root(graph):
            stack, queue, graph = shift(stack, queue, graph)
            return stack, queue, graph, 'sh'
        else:
            stack, queue, graph = left_arc(stack, queue, graph, trans[3:])
            return stack, queue, graph, 'la'
    if stack and can_reduce(stack, graph) and trans == 're':
        stack, queue, graph = reduce(stack, queue, graph)
        return stack, queue, graph, 're'
    stack, queue, graph = shift(stack, queue, graph)
    return stack, queue, graph, 'sh'

In [36]:
def extract_values(params):
    stack, queue, graph = params[0], params[1], params[2]
    return stack, queue, graph

In [54]:
def apply_trans_deep(stem, action):
    stack, queue, graph = extract_values(stem['params'])
    stack_c, queue_c, graph_c = copy.deepcopy(stack), copy.deepcopy(queue), copy.deepcopy(graph)
    
    return apply_transition(stack_c, queue_c, graph_c, action)

In [38]:
classes = classifier.classes_[0:]

def dictorize_prob_matrix(prob_matrix):
    return dict(zip(classes, prob_matrix))

def class_to_top_proba(prob_matrix_dict, depth, tool=1):
    sorted_prob = list(sorted(prob_matrix_dict.items(), key=lambda entry: -entry[1]))
    max_val = sorted_prob[0][1]
    
    filtred_prob = list(filter(lambda prob_tupel: prob_tupel[1] > max_val - tool, sorted_prob))
    
    return filtred_prob[0:depth]
        

In [39]:
def probabilites(depth, stack, queue, graph, sentence):
    X_test_dict = extract(depth, stack, queue, graph, sentence)
    X_test = vec.transform(X_test_dict)
    y_probabilites = classifier.predict_proba(X_test)[0]

    return dictorize_prob_matrix(y_probabilites)

In [52]:
def calculate(beam, branches, tool):
    done = True
    
    nbrBranches = len(branches)
    for nbr in range(nbrBranches): 
                stem = branches[nbr]
                stack, queue, graph = extract_values(stem['params'])

                dictorized_proba = probabilites(depth, stack, queue, graph, sentence)

                searchRange = beam - (nbrBranches - 1)
                top_proba = class_to_top_proba(dictorized_proba, searchRange, tool)

                if len(queue) > 0:
                    done = False

                    for action, prob_of_action in top_proba[1:]:
                        stack_n, queue_n, graph_n, trans = apply_trans_deep(stem, action)

                        sequence = stem['sequence'] + [trans]
                        total_prob = stem['score'] * prob_of_action
                        params = [stack_n, queue_n, graph_n, trans]

                        branch = {'params' : params, 'score' : total_prob, 'sequence' : sequence}                   
                        branches[nbrBranches] = branch

                        nbrBranches += 1;

                    action, prob_of_action = top_proba[0]

                    stack, queue, graph, trans = apply_transition(stack, queue, graph, action)

                    stem['sequence'].append(trans)
                    stem['score'] *= prob_of_action
                    stem['params'] = [stack, queue, graph, trans]
        
    return done, nbrBranches == beam

In [89]:
def assign_values_to(sentence, graph):
    for word in sentence:     
        word['HEAD'] = graph['heads'][word['ID']]
        word['DEPREL'] = graph['deprels'][word['ID']]

In [57]:
from tqdm import tqdm

beam = 2

tool = 0.5
graphs = []

for sent_cnt, sentence in tqdm(enumerate(formatted_corpus_test_clean)):
    X_test_dict = []
    
    stack, queue, graph = init_config(sentence)
    branches = {0 : {'params' : [stack, queue, graph], 'score' : 1, 'sequence' : []}}
    done = False
    
    while not done:
        done, trash = calculate(beam, branches, tool)
                    
    branch = max(branches.values(), key=lambda branch: branch['score'])
    params = branch['params']
    
    stack, graph = params[0], params[2] 
    stack, graph = empty_stack(stack, graph)
    graphs.append(graph)
    
    assign_values_to(sentence, graph)

1219it [02:08,  9.47it/s]


In [61]:
test1 = {0 : {'score' : 4}, 1 : {'score' : 2}, 2 : {'score' : 1}}

item = {'score' : 3}
print(insert(test1, item, 3))
print(test1)

test2 = {0 : {'score' : 4}, 1 : {'score' : 2}, 2 : {'score' : 1}}
item = {'score' : 3}
item['location'] = insert_full(test2, item, 3)
print(test2)

test3 = test2
update_and_remove(test3, item['location'], -1, direction=1)
print(test3)

test4 = {0 : {'score' : 4}, 1 : {'score' : 2}, 2 : {'score' : 1}}
t_l = [(0, 2), (1, 2)] 
all_tuples_higher_evaluated(test4, t_l, 1)

test5 = {0 : {'score' : 2}}
item = {'score' : 1}
insert(test5, item, 1)
print(test5)
test5[1]

NameError: name 'insert' is not defined

In [ ]:
def all_tuples_higher_evaluated(branches, t_list, current_score):
    max_index = len(branches) - 1
    accepted = []
    
    for tupel in t_list:
        total_prob = tupel[1]*current_score
        if  total_prob > branches[max_index]['score']:
            accepted.append(tupel)
            max_index -= 1
        else:
            break
            
    return accepted

def update_and_remove(dictionary, start, space, direction=-1):
    update(dictionary, start, space, direction)
    max_index = len(dictionary) - 1
    
    del dictionary[max_index]

def update(dictionary, start, space, direction=-1):
    length = len(dictionary) + space
    
    for i in reversed(range(start, length)):
        dictionary[i] = dictionary[i + direction]
def insert_full(dictionary, item, limit, space=0):
    index = -1
    
    for i, value in enumerate(dictionary.values()):
        if value['score'] < limit:
            update(dictionary, i + 1, space)
            dictionary[i] = item
            index = i
            return index
    if space == 1:
        length = len(dictionary)
        dictionary[length] = item
        return length
            
def insert(dictionary, item, limit):
    return insert_full(dictionary, item, limit, space=1)

In [121]:
from tqdm import tqdm

beam = 2
reach = 1
tool = 0.5
graphs = []

for sent_cnt, sentence in tqdm(enumerate(formatted_corpus_test_clean)):
    X_test_dict = []
    
    stack, queue, graph = init_config(sentence)
    branches = {0 : {'params' : [stack, queue, graph], 'score' : 1, 'sequence' : [], 'location' : 0}}
    
    done = False
    full = False
    
    while not done:
        if full:
            done = True
            
            nbrBranches = len(branches)
            results = []
            
            for nbr in range(nbrBranches):
                stem = branches[nbr]
                stack, queue, graph = extract_values(stem['params'])
                if len(queue) > 0:
                    done = False
                    dictorized_proba = probabilites(depth, stack, queue, graph, sentence)

                    top_proba = class_to_top_proba(dictorized_proba, 1 + reach)
                    results += list(map(lambda tupel: [nbr, tupel], top_proba))
                
            if not done:
                old_branches = copy.deepcopy(branches)
                sorted_results = sorted(results, key=lambda idx_tupel: -idx_tupel[1][1])

                for nbr in range(beam):
                    idx_tupel = sorted_results[nbr]

                    branch_id = idx_tupel[0]
                    action = idx_tupel[1][0]
                    prob_of_action = idx_tupel[1][1]


                    stem = old_branches[branch_id]
                    stack, queue, graph, trans = apply_trans_deep(stem, action)

                    sequence = stem['sequence'] + [trans]
                    total_prob = stem['score'] * prob_of_action
                    params = [stack, queue, graph, trans]

                    branch = {'params' : params, 'score' : total_prob, 'sequence' : sequence}  

                    branches[nbr] = branch

        else:
            done, full = calculate(beam, branches, tool)
    
    branch = max(branches.values(), key=lambda branch: branch['score'])            
    params = branch['params']
    
    stack, graph = params[0], params[2] 
    stack, graph = empty_stack(stack, graph)
    graphs.append(graph)
    
    assign_values_to(sentence, graph)

4it [00:00,  9.71it/s]

{'heads': {'0': '0', '2': '3', '1': '3', '6': '7', '5': '7', '4': '7', '3': '7', '8': '7', '9': '7', '10': '11', '11': '7', '12': '11', '13': '11', '14': '7', '15': '7', '7': '0'}, 'deprels': {'0': 'ROOT', '2': 'amod', '1': 'det', '6': 'nummod', '5': 'case', '4': 'cop', '3': 'nsubj', '8': 'punct', '9': 'nmod', '10': 'cc', '11': 'conj', '12': 'punct', '13': 'conj', '14': 'punct', '15': 'punct', '7': 'root'}}
{'heads': {'0': '0', '1': '2', '2': '0', '3': '2', '4': '2', '6': '7', '5': '7', '7': '2', '8': '2'}, 'deprels': {'0': 'ROOT', '1': 'nsubj', '2': 'root', '3': 'nsubj', '4': 'conj', '6': 'amod', '5': 'case', '7': 'obj', '8': 'punct'}}
{'heads': {'0': '0', '1': '2', '2': '0', '3': '4', '4': '2', '5': '6', '6': '4', '7': '6', '8': '4', '9': '4', '10': '2'}, 'deprels': {'0': 'ROOT', '1': 'nsubj:pass', '2': 'root', '3': 'case', '4': 'obl', '5': 'case', '6': 'nmod', '7': 'punct', '8': 'nmod', '9': 'punct', '10': 'punct'}}
{'heads': {'0': '0', '1': '2', '4': '5', '3': '5', '2': '5', '5': '

6it [00:00,  8.19it/s]

{'heads': {'0': '0', '1': '0', '2': '3', '3': '1', '4': '5', '5': '1', '6': '5', '7': '1', '8': '7', '9': '10', '10': '7', '11': '1'}, 'deprels': {'0': 'ROOT', '1': 'root', '2': 'case', '3': 'nmod', '4': 'case', '5': 'nmod', '6': 'nummod', '7': 'acl', '8': 'advmod', '9': 'case', '10': 'obl', '11': 'punct'}}
{'heads': {'0': '0', '4': '5', '3': '5', '5': '2', '2': '6', '1': '6', '8': '9', '7': '9', '9': '6', '11': '12', '12': '13', '10': '13', '13': '6', '15': '16', '16': '17', '14': '17', '17': '13', '19': '20', '18': '20', '20': '6', '21': '22', '22': '6', '23': '6', '6': '0'}, 'deprels': {'0': 'ROOT', '4': 'det', '3': 'case', '5': 'nmod', '2': 'nsubj', '1': 'mark', '8': 'amod', '7': 'det', '9': 'obj', '11': 'det', '12': 'nmod:poss', '10': 'case', '13': 'obj', '15': 'advmod', '16': 'nummod', '14': 'case', '17': 'nmod', '19': 'nsubj', '18': 'cop', '20': 'conj', '21': 'case', '22': 'obj', '23': 'punct', '6': 'root'}}
{'heads': {'0': '0', '2': '3', '3': '4', '1': '4', '5': '6', '6': '4', 

10it [00:01,  8.41it/s]

...

{'heads': {'0': '0', '1': '2', '2': '3', '3': '0', '4': '5', '5': '6', '6': '3', '9': '10', '8': '10', '7': '10', '10': '3', '13': '14', '15': '16', '14': '16', '12': '16', '11': '16', '16': '3', '17': '18', '18': '16', '19': '3'}, 'deprels': {'0': 'ROOT', '1': 'nmod:poss', '2': 'nsubj', '3': 'root', '4': 'advmod', '5': 'amod', '6': 'obj', '9': 'amod', '8': 'det', '7': 'case', '10': 'obl', '13': 'det', '15': 'aux', '14': 'nsubj', '12': 'cc', '11': 'punct', '16': 'conj', '17': 'case', '18': 'obl', '19': 'punct'}}


1217it [03:29,  4.74it/s]

{'heads': {'0': '0', '3': '4', '2': '4', '1': '4', '4': '0', '5': '4', '8': '7', '7': '9', '6': '9', '9': '4', '10': '9', '11': '12', '12': '10', '16': '17', '15': '17', '14': '17', '13': '17', '17': '12', '19': '20', '18': '20', '20': '17', '21': '4'}, 'deprels': {'0': 'ROOT', '3': 'advmod', '2': 'cop', '1': 'expl', '4': 'root', '5': 'nsubj', '8': 'fixed', '7': 'advmod', '6': 'mark', '9': 'csubj', '10': 'obj', '11': 'case', '12': 'nmod', '16': 'det', '15': 'cop', '14': 'nsubj', '13': 'punct', '17': 'conj', '19': 'amod', '18': 'case', '20': 'obl', '21': 'punct'}}


1219it [03:29,  5.81it/s]

{'heads': {'0': '0', '2': '3', '1': '3', '4': '5', '5': '3', '6': '7', '7': '3', '8': '9', '3': '9', '9': '0', '10': '9', '11': '12', '12': '10', '13': '14', '14': '12', '15': '16', '16': '9', '18': '19', '17': '19', '19': '16', '20': '21', '21': '19', '22': '23', '23': '16', '24': '25', '25': '23', '27': '28', '26': '28', '28': '23', '29': '9'}, 'deprels': {'0': 'ROOT', '2': 'amod', '1': 'det', '4': 'case', '5': 'nmod', '6': 'cc', '7': 'nmod', '8': 'aux', '3': 'nsubj', '9': 'root', '10': 'obj', '11': 'case', '12': 'nmod', '13': 'punct', '14': 'nmod', '15': 'aux', '16': 'advcl', '18': 'amod', '17': 'det', '19': 'obl', '20': 'cc', '21': 'conj', '22': 'aux', '23': 'advcl', '24': 'amod', '25': 'obj', '27': 'det', '26': 'case', '28': 'xcomp', '29': 'punct'}}
{'heads': {'0': '0', '3': '4', '2': '4', '1': '4', '4': '0', '5': '4', '6': '7', '7': '5', '8': '4', '10': '11', '9': '11', '11': '4', '12': '4'}, 'deprels': {'0': 'ROOT', '3': 'nsubj', '2': 'aux', '1': 'advmod', '4': 'root', '5': 'obj

In [123]:
graph_collection_errors(clean_test, graphs, cond_deprel=True)

[{'ID': '0',
  'FORM': 'root',
  'LEMMA': 'root',
  'UPOS': 'root',
  'XPOS': 'root',
  'FEATS': 'root',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': 'root',
  'MISC': 'root'},
 {'ID': '1',
  'FORM': 'Siffrorna',
  'LEMMA': 'siffra',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|PLU|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Gender=Com|Number=Plur',
  'HEAD': '3',
  'DEPREL': 'nsubj:pass',
  'DEPS': '3:nsubj:pass',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'kan',
  'LEMMA': 'kunna',
  'UPOS': 'AUX',
  'XPOS': 'VB|PRS|AKT',
  'FEATS': 'Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act',
  'HEAD': '3',
  'DEPREL': 'aux',
  'DEPS': '3:aux',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'ställas',
  'LEMMA': 'ställa',
  'UPOS': 'VERB',
  'XPOS': 'VB|INF|SFO',
  'FEATS': 'VerbForm=Inf|Voice=Pass',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': '0:root',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'under',
  'LEMMA': 'under',
  'UPOS': 'ADV',
  'XPOS': 'AB',
  'FEATS': '_',
  'HEAD': '3',
  'DEPREL': 'compound:prt',
  'DEPS

In [230]:
formatted_corpus_test_clean[8]

[{'ID': '0',
  'FORM': 'root',
  'LEMMA': 'root',
  'UPOS': 'root',
  'XPOS': 'root',
  'FEATS': 'root',
  'HEAD': '0',
  'DEPREL': 'ROOT',
  'DEPS': 'root',
  'MISC': 'root'},
 {'ID': '1',
  'FORM': 'Allt',
  'LEMMA': 'all',
  'UPOS': 'ADV',
  'XPOS': 'AB',
  'FEATS': '_',
  'HEAD': '2',
  'DEPREL': 'advmod',
  'DEPS': '5:mark',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'efter',
  'LEMMA': 'efter',
  'UPOS': 'ADP',
  'XPOS': 'PP',
  'FEATS': '_',
  'HEAD': '4',
  'DEPREL': 'case',
  'DEPS': '1:fixed',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'som',
  'LEMMA': 'som',
  'UPOS': 'ADV',
  'XPOS': 'HA',
  'FEATS': '_',
  'HEAD': '4',
  'DEPREL': 'advmod',
  'DEPS': '1:fixed',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'prisläget',
  'LEMMA': 'prisläge',
  'UPOS': 'NOUN',
  'XPOS': 'NN|NEU|SIN|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Gender=Neut|Number=Sing',
  'HEAD': '5',
  'DEPREL': 'nsubj:pass',
  'DEPS': '5:nsubj:pass',
  'MISC': '_'},
 {'ID': '5',
  'FORM': 'förändras',
  'LEMMA': 'förändra

In [ ]:
formatted_corpus_test[0]

In [ ]:
len(formatted_corpus_test_clean)

In [91]:
def save(file, formatted_corpus, column_names):
    f_out = open(file, 'w', encoding='utf-8')
    for sentence in formatted_corpus:
        for row in sentence[1:]:
            # print(row, flush=True)
            for col in column_names[:-1]:
                if col in row:
                    f_out.write(row[col] + '\t')
                else:
                    f_out.write('_\t')
            col = column_names[-1]
            if col in row:
                f_out.write(row[col] + '\n')
            else:
                f_out.write('_\n')
        f_out.write('\n')
    #f_out.write('\n')
    f_out.close()

In [92]:
out_file_name = 'out_sys'
save(out_file_name, formatted_corpus_test_clean, column_names_u)

### Evaluation

1. Once you have parsed the test set, you will measure the accuracy of your parser using the CoNLL evaluation script available here: http://universaldependencies.org/conll18/evaluation.html. Download it.
2. You will run the evaluation in the cell below. Be sure to have the Python program in your folder.
2. You will report your best score. You need to reach a _labelled attachment score_ (LAS) of 0.67 to pass this lab.

In [94]:
import conll18_ud_eval
system_ud_file = open(out_file_name, encoding='utf-8')
system_ud = conll18_ud_eval.load_conllu(system_ud_file)

gold_ud_file = open(path_sv_test, encoding='utf-8')
gold_ud = conll18_ud_eval.load_conllu(gold_ud_file)

las = conll18_ud_eval.evaluate(gold_ud, system_ud)['LAS'].f1
las

0.6654561515434068

Should you want to run the script in other experiments, just execute: `python conll18_ud_eval.py gold_file system_file`

### Reading

Read the article: _Globally Normalized Transition-Based Neural Networks_ by Andor and al. (2016) [<a href="https://www.aclweb.org/anthology/P16-1231">pdf</a>] and write in a few sentences how it relates to your work in this assignment.</p>

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [ ]:
STIL_ID = ["vi1146ol-s"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "c-dependency_parsing.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of your best labeled attachment score.

In [147]:
import json
ANSWER = json.dumps({'las': las
                    })
ANSWER

'{"las": 0.6934779408156254}'

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [148]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [149]:
import bz2
ASSIGNMENT = 6
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [150]:
import requests
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
               verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': '64dae10e41f52c356dbb4c537da5e4931d6c1011b3a318db36dd4e4fb3c0d8d14f329c73b220be06b224affaef93761c9c5daf440e08f7633a3e024f6f235c92',
 'submission_id': 'd6c2de47-2d14-4ef2-97f6-8b115b39b32a'}